# P19 Benchmark

### Import modules

In [1]:
import os
from datetime import datetime
from pathlib import Path

import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
from torcheval.metrics import BinaryAUROC, BinaryAUPRC

os.chdir('..')
from raindrop.raindrop import Raindrop
from util.grad_track import GradientTracker

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
PROCESSED_PATH = Path('./p19/data/processed_data')
static_feat_names = np.load(PROCESSED_PATH / 'labels_demogr.npy')
ts_feat_names = np.load(PROCESSED_PATH / 'labels_ts.npy')
inputs = np.load(PROCESSED_PATH / 'PT_dict_list_6.npy', allow_pickle=True)
labels = np.load(PROCESSED_PATH / 'arr_outcomes_6.npy').squeeze()

ts_inputs = np.array([inp['arr'] for inp in inputs])[:, :, :, np.newaxis]
static_inputs = np.array([inp['extended_static'] for inp in inputs])
times = np.array([inp['time'] for inp in inputs]).squeeze()
lengths = np.array([inp['length'] for inp in inputs])

In [4]:
def oversample(ts_inputs, static_inputs, times, lengths, labels):
    neg, pos = ~labels.astype(bool), labels.astype(bool)
    num_neg, num_pos = sum(neg), sum(pos)
    fill_to = num_neg
    
    def oversample_array(array):
        arr_neg, arr_pos = array[neg], array[pos]
        
        arr_pos = np.repeat(arr_pos, fill_to // num_pos, axis=0)
        if (rem := fill_to % num_pos):
            arr_pos = np.concatenate([arr_pos, arr_pos[:rem]], axis=0)

        return np.concatenate([arr_pos, arr_neg], axis=0)
    
    ts_inputs = oversample_array(ts_inputs)
    static_inputs = oversample_array(static_inputs)
    times = oversample_array(times)
    lengths = oversample_array(lengths)
    labels = oversample_array(labels)
    
    return ts_inputs, static_inputs, times, lengths, labels
    
ts_inputs, static_inputs, times, lengths, labels = oversample(ts_inputs, static_inputs, times, lengths, labels)

In [5]:
ts_inputs = torch.tensor(ts_inputs, dtype=torch.float32).to(device)
static_inputs = torch.tensor(static_inputs, dtype=torch.float32).to(device)
times = torch.tensor(times, dtype=torch.float32).to(device)
lengths = torch.tensor(lengths).to(device)
labels = torch.tensor(labels, dtype=torch.int64).to(device)

### Split data

In [6]:
np.random.seed(42)

num_samples = ts_inputs.shape[0]
idxs = np.arange(num_samples)
np.random.shuffle(idxs)

train_idxs, val_idxs, test_idxs = idxs[:(s1 := int(num_samples*0.8))], idxs[s1: (s2 := int(num_samples*0.9))], idxs[s2:]

train_ts_inp, val_ts_inp, test_ts_inp = ts_inputs[train_idxs], ts_inputs[val_idxs], ts_inputs[test_idxs]
train_static_inp, val_static_inp, test_static_inp = static_inputs[train_idxs], static_inputs[val_idxs], static_inputs[test_idxs]
train_times, val_times, test_times = times[train_idxs], times[val_idxs], times[test_idxs]
train_lengths, val_lengths, test_lengths = lengths[train_idxs], lengths[val_idxs], lengths[test_idxs]
train_lbls, val_lbls, test_lbls = labels[train_idxs], labels[val_idxs], labels[test_idxs]

In [7]:
print("P19 Summary")
print(f"Num Samples: {num_samples}\n\tTrain: {train_idxs.shape[0]}\n\tVal: {val_idxs.shape[0]}\n\tTest: {test_idxs.shape[0]}")

pos = [(int(t := labels.sum()), 100 * t / labels.shape[0])] + \
      [(t := int(lbls.sum()), 100 * t / lbls.shape[0]) for lbls in (train_lbls, val_lbls, test_lbls)]
print(f"Classes: {pos[0][0]} {pos[0][1]:.2f}%")
for lbl, (t, p) in zip(['Train', 'Val', 'Test'], pos):
    print(f"\t{lbl}: {t} {p:.2f}%")

P19 Summary
Num Samples: 74354
	Train: 59483
	Val: 7435
	Test: 7436
Classes: 37177 50.00%
	Train: 37177 50.00%
	Val: 29659 49.86%
	Test: 3684 49.55%


In [8]:
from torch.utils.data import Dataset, DataLoader
from torch import Tensor

class P19Dataset(Dataset):
    def __init__(self,
                 ts_inp: Tensor,
                 times: Tensor,
                 lengths: Tensor,
                 static_inp: Tensor,
                 labels: Tensor,
                 device: str):
        self.ts_inp = ts_inp
        self.times = times
        self.lengths = lengths
        self.static_inp = static_inp
        self.labels = labels
        self.device = device

    def __len__(self) -> int:
        return len(self.labels)
    
    def __getitem__(self, loc: int | slice):
        ts_inp = self.ts_inp[loc]

        # Create mask
        mask = torch.zeros(ts_inp.shape[:-1], dtype=bool, device=self.device)
        if isinstance(loc, int):
            mask[:self.lengths[loc]] = 1
        else:
            for idx, length in enumerate(self.lengths[loc]):
                mask[idx, :length] = 1

        return ts_inp, self.times[loc], mask, self.static_inp[loc], self.labels[loc]

In [9]:
train_ds = P19Dataset(train_ts_inp, train_times, train_lengths, train_static_inp, train_lbls, device)
val_ds = P19Dataset(val_ts_inp, val_times, val_lengths, val_static_inp, val_lbls, device)
test_ds = P19Dataset(test_ts_inp, test_times, test_lengths, test_static_inp, test_lbls, device)

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=128, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=128, shuffle=True)


### Define the model

In [10]:
raindrop = Raindrop(num_sensors=34,
                 obs_dim=1,
                 obs_embed_dim=4,
                 pe_emb_dim=16,
                 timesteps=60,
                 out_dim=128,
                 num_heads=1,
                 num_layers=1,
                 inter_sensor_attn_dim=16,
                 temporal_attn_dim=16,
                 prune_rate=0.5,
                 device=device)



class RaindropClassifier(nn.Module):
    def __init__(self,
                 rd_model: Raindrop,
                 static_dim: int,
                 static_proj_dim: int,
                 cls_hidden_dim: int,
                 classes: int):
        super().__init__()
        self.static_dim = static_dim
        self.static_proj_dim = static_proj_dim
        self.cls_hidden_dim = cls_hidden_dim
        self.classes = classes

        self.rd_model = rd_model
        self.static_proj = nn.Linear(static_dim, static_proj_dim)

        rd_out_dim = self.rd_model.out_dim * self.rd_model.num_sensors
        self.cls = nn.Sequential(
            nn.Linear(rd_out_dim + static_proj_dim, cls_hidden_dim),
            nn.LeakyReLU(),
            nn.Linear(cls_hidden_dim, classes),
            nn.LeakyReLU())


    def forward(self, x_ts, times, mask, x_static):
        ts_emb, reg_loss = self.rd_model(x_ts, times, mask)
        ts_emb = ts_emb.view(ts_emb.shape[0], -1)

        static_emb = F.leaky_relu(self.static_proj(x_static))

        emb = torch.concat([ts_emb, static_emb], dim=-1) 
        return F.softmax(self.cls(emb), dim=-1), reg_loss
    
rd_cls = RaindropClassifier(raindrop,
                            static_dim=6,
                            static_proj_dim=34,
                            cls_hidden_dim=128,
                            classes=2).to(device)


### Training utilities

In [11]:

class RaindropLoss(nn.Module):
    def __init__(self, reg_weight: float):
        super().__init__()
        
        self.ce_loss = nn.CrossEntropyLoss()
        self.reg_weight = reg_weight

    def forward(self, predictions, targets, reg_loss):
        ce_loss = self.ce_loss(predictions, targets)
        # reg_loss *= self.reg_weight
        # return ce_loss + reg_loss
        return ce_loss
    
loss_fn = RaindropLoss(reg_weight=0.02)
# loss_fn = nn.CrossEntropyLoss()

In [12]:
NUM_EPOCHS = 20
LOSS_TRAIN_LOG_FREQ = 100

optim = torch.optim.Adam(rd_cls.parameters(), lr=0.0001)

bin_auroc_metric = BinaryAUROC()
bin_auprc_metric = BinaryAUPRC()

In [13]:
# gt = GradientTracker(rd_cls)

### Train loop

In [14]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss, running_auroc, running_auprc = 0., 0., 0.
    last_loss, last_auroc, last_auprc = 0., 0., 0.

    for i, data in enumerate(train_dl):
        ts_inp, times, mask, static_inp, labels = data

        optim.zero_grad()
        outputs, reg_loss = rd_cls(ts_inp, times, mask, static_inp)
        loss = loss_fn(outputs, labels, reg_loss)
        # loss = loss_fn(outputs, labels)
        bin_outputs = outputs.argmax(dim=-1)
        bin_auroc_metric.update(bin_outputs, labels)
        bin_auprc_metric.update(bin_outputs, labels)
        loss.backward()

        optim.step()

        running_loss += loss.item()
        running_auroc += bin_auroc_metric.compute().item()
        running_auprc += bin_auprc_metric.compute().item()

        if i % LOSS_TRAIN_LOG_FREQ == LOSS_TRAIN_LOG_FREQ-1:
            # return 
            last_loss = running_loss / LOSS_TRAIN_LOG_FREQ
            last_auroc = running_auroc / LOSS_TRAIN_LOG_FREQ
            last_auprc = running_auprc / LOSS_TRAIN_LOG_FREQ
    
            print('  batch {} loss: {} AUROC: {} AUPRC {}'.format(i + 1, last_loss, last_auroc, last_auprc))
            
            tb_x = epoch_index * len(train_dl) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            tb_writer.add_scalar('AUROC/train', last_auroc, tb_x)
            tb_writer.add_scalar('AUPRC/train', last_auprc, tb_x)

            running_loss, running_auroc, running_auprc = 0., 0., 0.

    return last_loss, last_auroc, last_auprc

In [15]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/p19/p19_trainer_{}'.format(timestamp))
epoch_number = 0

best_vloss = 1e6

for epoch in range(NUM_EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    rd_cls.train(True)
    avg_loss, avg_auroc, avg_auprc = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    running_vloss, running_vauroc, running_vauprc = 0., 0., 0.
    rd_cls.eval()

    with torch.no_grad():
        for i, vdata in enumerate(val_dl):
            ts_inp, times, mask, static_inp, labels = vdata
            voutputs, reg_loss = rd_cls(ts_inp, times, mask, static_inp)
            vloss = loss_fn(voutputs, labels, reg_loss)
            # vloss = loss_fn(voutputs, labels)
            bin_voutputs = voutputs.argmax(dim=-1)
            bin_auroc_metric.update(bin_voutputs, labels)
            bin_auprc_metric.update(bin_voutputs, labels)

            running_vloss += vloss
            running_vauroc += bin_auroc_metric.compute()
            running_vauprc += bin_auprc_metric.compute()

    avg_vloss = running_vloss / (i + 1)
    avg_vauroc = running_vauroc / (i + 1)
    avg_vauprc = running_vauprc / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation AUROC',
                    { 'Training' : avg_auroc, 'Validation' : avg_vauroc },
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation AUPRC',
                    { 'Training' : avg_auprc, 'Validation' : avg_vauprc },
                    epoch_number + 1)

    writer.flush()

    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(rd_cls.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
  batch 100 loss: 0.5808805045485497 AUROC: 0.6955615089275227 AUPRC 0.6407318341732026
  batch 200 loss: 0.581168327331543 AUROC: 0.7135721607385856 AUPRC 0.6655090445280075
  batch 300 loss: 0.5790930593013763 AUROC: 0.7144236927343147 AUPRC 0.6677868002653122
  batch 400 loss: 0.5700709235668182 AUROC: 0.7153809397753207 AUPRC 0.6677020889520645
LOSS train 0.5700709235668182 valid 0.5663967728614807
EPOCH 2:
  batch 100 loss: 0.5678902086615563 AUROC: 0.7203547295172069 AUPRC 0.672495938539505
  batch 200 loss: 0.5655494064092637 AUROC: 0.7213465432113333 AUPRC 0.6737380599975586
  batch 300 loss: 0.5533120912313462 AUROC: 0.7237033848340998 AUPRC 0.6762548834085464
  batch 400 loss: 0.5539217674732209 AUROC: 0.7260796801748255 AUPRC 0.6785876822471618
LOSS train 0.5539217674732209 valid 0.5506409406661987
EPOCH 3:
  batch 100 loss: 0.5466507241129875 AUROC: 0.7304866750830736 AUPRC 0.6829441636800766
  batch 200 loss: 0.5446849721670151 AUROC: 0.7321095898429532 AUPRC 0.68